In [ ]:
import numpy as np
import pandas as pd
import pathlib
import fnmatch
import datetime

In [2]:
#This cell takes the path to the folder, which contains !!UNZIPPED!! folders, which contain one weeks homeworks.
#The folders corresponding to weeks conform to the naming scheme of Zip files, but !!MANUAL UNZIPPING IS REQUIRED!!.

#Unizipping via code seemed like a waste of time, compared to right-clicking and selecting "Extract All". :=)
#The output is a dataframe, which feeds into the code in the next cell.
#I am sure there is a more elegant way to do this, but this works.


pth = ... #YOUR PATH HERE


def studentPaths(pth):
    pathlist1 = []
    for p in pathlib.Path(pth).iterdir():
        pathlist1.append(str(p))
    return pathlist1
ehe = studentPaths(pth)

def homePaths(ehe):
    pathlist2 = []
    for element in ehe:
        if element.endswith(".ceg"):
            ehe.remove(element)
            continue
        if pathlib.Path(element).is_file():
            continue
        for p in pathlib.Path(element).iterdir():
            pathlist2.append(p)
    return pathlist2
ehe2 = homePaths(ehe)
def filterbyStudent(ehe2):
    pathlist3 = []
    pusnus = []
    for element in ehe2:
        if pathlib.Path(element).is_file():
            continue
        for p in pathlib.Path(element).iterdir():
            pathlist3.append(p)
    for element in pathlist3:
        eletxt = str(element).split("\\")
        temp = len(eletxt)
        esitus = eletxt[temp-1]
        õpilane = eletxt[temp-2]
        kodutöö = eletxt[temp-3]
        if esitus.endswith(".ceg") == False:
            pusnus.append((õpilane, kodutöö, esitus, element))
        panda = pd.DataFrame(pusnus, columns=["Õpilane", "Kodutöö", "Esitus", "Path"])
    return panda
panda = filterbyStudent(ehe2)

In [3]:
#In this cell, the dataframe is cleaned from submissions, that are not the latest.
#The output is yet another dataframe, which is used in the next cell.
#Using multiple data frames is not very efficient, but was helpful for debugging.
df1 = panda.groupby(['Õpilane', 'Kodutöö', "Path"])['Esitus'].apply(list).reset_index()
for index, row in df1.iterrows():
    if row["Õpilane"].startswith("K"):
        df1.drop(labels=None, axis=0, index=index, columns=None, level=None, inplace=True, errors='raise')
        continue
    #This chunk of code doesn't work perfectly, but it's needed for CleanDuplicates to work.
    datetime_object = datetime.datetime(1900, 1, 1, 0, 0, 0)
    for element in row['Esitus']:
        datetime_objectiter = datetime.datetime.strptime(element, '%Y-%m-%d-%H-%M-%S')
        if datetime_objectiter > datetime_object:
            datetime_object = datetime_objectiter
    df1.at[index, 'Esitus'] = element
    df1.at[index, "EsitusDT"] = datetime_object
    #End of aforementioned chunk of code.
for index, row in df1.iterrows():
    pathbits = str(row["Path"]).split("\\")
    pathbits.pop()
    paff = "\\".join(pathbits)
    df1.at[index, 'Path'] = str(paff) + "\\" + str(row['Esitus'])
    df1.at[index, 'Path'] = pathlib.Path(row["Path"])

def CleanDuplicates(df1):
    df1 = df1.sort_values(by=["EsitusDT"], ascending=True).drop_duplicates(subset=['Õpilane', 'Kodutöö'], keep='last')
    return df1
df1 = CleanDuplicates(df1)          

In [4]:
#In this cell, the final dataframe is created. It contains all the latest submissions of all students.
#There are 4 columns, one for the homework week, one for the student, one for the task and one for the text, in that order.
finallist = []
for idx, row in df1.iterrows():
    for file in pathlib.Path(row["Path"]).iterdir():
        if fnmatch.fnmatch(file, "*.py"):
            filenamebits = str(file).split("\\")
            temp = len(filenamebits)
            filename = filenamebits[temp-1]
            f = open(file, "r", encoding="utf-8")
            text = f.read()
            finallist.append((row["Kodutöö"], row["Õpilane"],  filename, text))
            f.close()
df4 = pd.DataFrame(finallist, columns=["Kodutöö", "Õpilane", "Ülesanne", "Tekst"])
df4 = df4.sort_values(by=["Kodutöö", "Õpilane", "Ülesanne"])


In [5]:
#Dataframe is grouped and exported
df4.groupby(["Kodutöö", "Ülesanne"])
df4.to_csv("C:\\Users\\JuuPumal\\Downloads\\Test\\Masinõpe\\df45.csv", encoding="utf-8", index=False, sep=";")

In [7]:
#Due to issues with duplicates, this dataframe displays the number of submissions per student per task.
#The greatest possible number of submissions from 1 student, for all the weeks, is 13 per task.

#In reality, the maximum number is 12, for task 1, since for week 2, the counting starts from kodu2.
#Task 2 goes up to 11, since week 1 only has 1 task and in week 8, kodu2 is called film instead.
#Task 3 and 4 are lower, since they are not present for every week.

#Considering the above, .describe is only used to check for the maximum value. Mean and median are not very informative, due to the presence of outliers (film appears only once etc.).
df_sanity = df4.groupby("Õpilane").value_counts(["Ülesanne"])
df_sanity.to_csv("C:\\Users\\JuuPumal\\Downloads\\Test\\Masinõpe\\df_sanity.csv", encoding="utf-8", index=True, sep=";")
df_sanity.describe()

count    1761.000000
mean        5.466780
std         4.029882
min         1.000000
25%         2.000000
50%         5.000000
75%         9.000000
max        12.000000
dtype: float64